# Evapotranspiration - exercises222

- toc: true 
- badges: true
- comments: false
- categories: [jupyter]

We will calculate the evapotranspiration using two methods: Thornthwaite and Penman.

## Download data from the IMS

Go to the Israel Meteorological Service website, and download the following data:
1. [hourly data](https://ims.data.gov.il/he/ims/1)
 * on the first page, choose all options and press continue.
 * on the next page, choose the following date range: 01/01/2020 to 01/01/2021, then press continue.
 * Choose station Bet Dagan (בית דגן 2523), then select (בחר), then continue.
 * Choose option "by station" (לפי תחנות), then produce report.
 * Download report as csv, call it "bet-dagan-3h.csv".
2. [daily data](https://ims.data.gov.il/he/ims/2)
 * on the first page, choose all options and press continue.
 * on the next page, choose the following date range: 01/01/2020 to 01/01/2021, then press continue.
 * Choose station Bet Dagan Meuyeshet (בית דגן מאוישת 2520), then select (בחר), then continue.
 * Choose option "by station" (לפי תחנות), then produce report.
 * Download report as csv, call it "bet-dagan-day-pan.csv".
3. [radiation data](https://ims.data.gov.il/he/ims/6)
 * on the first page, choose all options, **then on the bottom right option "radiation" (קרינה), choose kJ/m2**, and then press continue.
 * on the next page, choose the following date range: 01/01/2020 to 01/01/2021, then press continue.
 * Choose station Bet Dagan Krina (בית דגן קרינה 2524), then select (בחר), then continue.
 * Choose option "by station" (לפי תחנות), then produce report.
 * Download report as csv, call it "bet-dagan-radiation.csv".
 
## Import relevant packages 

In [1]:
#collapse-hide

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()  # datetime converter for a matplotlib
import seaborn as sns
sns.set(style="ticks", font_scale=1.5)

## import hourly data

In [2]:
#collapse-hide

df = pd.read_csv('bet-dagan-3h.csv', encoding = 'unicode_escape', na_values=["-"])
# find out what hebrew gibberish means: http://www.pixiesoft.com/flip/
name_conversion_dictionary = {"ùí úçðä": "station name",
                              "îñôø úçðä": "station number",
                              "úàøéê": "Date",
                              "ùòä-LST": "LST time",
                              "èîôøèåøä(C°)": "T",
                              "èîôøèåøä ìçä(C°)": "wet-bulb temperature (°C)",
                              "èîôøèåøú ð÷åãú äèì(C°)": "dew_point_T",
                              "ìçåú éçñéú(%)": "relative humidity (%)",
                              "îäéøåú äøåç(m/s)": "wind_speed",
                              "ëéååï äøåç(îòìåú)": "wind direction (degrees)",
                              "ìçõ áâåáä äúçðä(hPa)": "Pressure",
                              "ìçõ áâåáä ôðé äéí(hPa)": "pressure at sea level (hPa)",
                              """äúàãåú éåîéú îâéâéú ñåâ à'(î"î)""": "pan evaporation (mm)",
                              "ñåâ ÷øéðä()": "radiation type",
                             }
# units
# T = temperature (°C)
# dew_point_T = dew point temperature (°C)
# wind_speed = wind speed (m/s)
# Pressure = pressure at station height (hPa = 0.1 kPa)

df = df.rename(columns=name_conversion_dictionary)
df['timestamp'] = df['Date'] + ' ' + df['LST time']
df['timestamp'] = pd.to_datetime(df['timestamp'], dayfirst=True)
df = df.set_index('timestamp')
df

,station name,station number,Date,LST time,T,wet-bulb temperature (°C),dew_point_T,relative humidity (%),wind_speed,wind direction (degrees),...,pressure at sea level (hPa),ëîåú òððéí ëåììú(÷åã),ëîåú òððéí ðîåëéí(÷åã),âåáä áñéñ òððéí ðîåëéí(÷åã),ñåâ äòððéí äðîåëéí(÷åã),ñåâ äòððéí äáéðåðééí(÷åã),ñåâ äòððéí äâáåäéí(÷åã),îæâ àååéø ðåëçé(÷åã),îæâ àååéø ùçìó(÷åã),øàåú àô÷éú(÷åã)
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 02:00:00,áéú ãâï ...,2523,01-01-2020,02:00,7.9,7.2,6.4,90,1.7,117.0,...,1018.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 05:00:00,áéú ãâï ...,2523,01-01-2020,05:00,7.5,7.0,6.4,93,1.2,116.0,...,1018.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 08:00:00,áéú ãâï ...,2523,01-01-2020,08:00,8.6,8.3,8.0,96,1.1,107.0,...,1018.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 11:00:00,áéú ãâï ...,2523,01-01-2020,11:00,15.9,13.1,10.6,71,2.4,196.0,...,1017.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 14:00:00,áéú ãâï ...,2523,01-01-2020,14:00,18.1,14.0,10.4,61,2.8,264.0,...,1015.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 11:00:00,áéú ãâï ...,2523,31-01-2021,11:00,19.0,13.7,8.9,52,5.6,235.0,...,1017.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-31 14:00:00,áéú ãâï ...,2523,31-01-2021,14:00,19.2,14.7,11.0,59,4.6,252.0,...,1016.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-31 17:00:00,áéú ãâï ...,2523,31-01-2021,17:00,18.2,14.8,12.2,68,0.8,203.0,...,1017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Part 1: Thornthwaite estimation

$$
E = 16\left[ \frac{10\,T^\text{monthly mean}}{I} \right]^a,
$$

where

$$
I = \sum_{i=1}^{12} \left[ \frac{T_i^\text{monthly mean}}{5} \right]^{1.514},
$$

and

$$
\begin{align}
a &= 6.75\times 10^{-7}I^3 \\
   &- 7.71\times 10^{-5}I^2 \nonumber\\
   &+ 1.792\times 10^{-2}I \nonumber\\
   &+ 0.49239 \nonumber
\end{align}
$$

 - $E$ is the monthly potential ET (mm)
 - $T_\text{monthly mean}$ is the mean monthly temperature in °C
 - $I$ is a heat index
 - $a$ is a location-dependent coefficient

From df, make a new dataframe, df_th, that stores monthly temperatures means. Use `resample` function.